In [29]:
# imports
import pandas as pd
import torch
from chess import pgn
import io
import chess
from tqdm.notebook import tqdm

In [30]:
path = '/Users/anu/Documents/GM_games_dataset.csv'

In [ ]:
df = pd.read_csv(path, nrows=230000, usecols=['pgn'])


In [32]:
# d = [king, queen, rook, bishop, knight, pawn, black, white, can_move]
def pgn_to_boardrep(game_pgn):
    game = pgn.read_game(game_pgn)
    moves = game.mainline_moves()
    board = chess.Board()
    pieces = chess.PIECE_TYPES
    states_arr = []
    moves_arr = []
    
    for move in moves:
        squares_arr = []
        for row in range(8):
            row_arr = []
            for col in range(8):
                square = (8 * row) + col
                col_arr = ([int(piece == board.piece_type_at(square)) for piece in pieces] + 
                [int(color == board.color_at(square)) for color in chess.COLORS]) + [int(board.turn == board.color_at(square))]
                row_arr.append(col_arr)
            squares_arr.append(row_arr)
        
        moves_arr.append([move.from_square, move.to_square])
        
        
        board.push(move)
        
        states_arr.append(squares_arr)

        
    return (torch.Tensor(states_arr, device='cpu'), torch.Tensor(moves_arr, device='cpu'))

In [33]:
features = torch.Tensor()
labels = torch.Tensor()
games = tqdm(df['pgn'].items(), total=len(df))
for game_pgn in games:
    features = torch.cat((features, pgn_to_boardrep(io.StringIO(game_pgn[1]))[0]))
    labels = torch.cat((labels, pgn_to_boardrep(io.StringIO(game_pgn[1]))[1]))

  0%|          | 0/20000 [00:00<?, ?it/s]

In [34]:
torch.save((features, labels), 'data/dataset.pt')

In [41]:
torch.load('data/dataset.pt')[0][0]

/var/folders/2b/d0h5v9t55gg2wl7c9csv2x280000gn/T/ipykernel_79736/102046578.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('data/dataset.pt')[0][0]


tensor([[[0., 0., 0., 1., 0., 0., 1., 0., 1.],
         [0., 1., 0., 0., 0., 0., 1., 0., 1.],
         [0., 0., 1., 0., 0., 0., 1., 0., 1.],
         [0., 0., 0., 0., 1., 0., 1., 0., 1.],
         [0., 0., 0., 0., 0., 1., 1., 0., 1.],
         [0., 0., 1., 0., 0., 0., 1., 0., 1.],
         [0., 1., 0., 0., 0., 0., 1., 0., 1.],
         [0., 0., 0., 1., 0., 0., 1., 0., 1.]],

        [[1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         